In [1]:
!pip install phonenumbers

In [8]:
import threading
import pandas as pd
import csv
import time
import re
import os

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities

import phonenumbers
from phonenumbers import PhoneNumberMatcher

In [1]:
import threading
import pandas as pd

class SharedListURLs:
    def __init__(self):
        self.allURLs = ['http://hespress.com/',
                    'https://www.hibapress.com',
                    'https://www.goud.ma',
                    'http://lemonde.fr/',
                    'https://www.lematin.ma'
                    ]
        self.data = []
        self.lock = threading.Lock()
        self.nb_URLs=0
        
    def load_data(self) :
        file_xslx_name = "./data/WebSites.xlsx"

        def url_processing(url):
            if url.startswith("http//"):
                return url.replace("http//", "http://")
            elif url.startswith("https//"):
                return url.replace("https//", "https://")
            elif not url.startswith("http://") and not url.startswith("https://"):
                return "https://" + url
            else:
                return url

        df = pd.read_excel(file_xslx_name)
        
        df['url'] = df['domain'].apply(url_processing)

        self.allURLs = df['url'].tolist()
        self.nb_URLs = len(self.allURLs)
        
    def select_data(self, start, end):
        if end < self.nb_URLs:
            self.data = self.allURLs[start:end]
        else:
            self.data = self.allURLs[start:self.nb_URLs]

In [1]:
import re
import phonenumbers
from phonenumbers import PhoneNumberMatcher
# import ContactItem

class ContactProcessing:
            
    def extract_contact_information(self, contenu_html, _url, contact_link_inf = None):
        try:
            pattern_email = re.compile(r'[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}')
            _emails = pattern_email.findall(contenu_html)
            _emails = set(_emails)

            telephones = set()
            matches = PhoneNumberMatcher(contenu_html, "MA")
            for match in matches:
                phone_number = match.number
                formatted_number = phonenumbers.format_number(phone_number, phonenumbers.PhoneNumberFormat.E164)
                telephones.add(formatted_number)

            social_networks = ['facebook','twitter','instagram','linkedin','youtube',"tiktok"]
            social_network_links = {}

            for network in social_networks: 
                pattern_network = re.compile(f'<a .*href="(https?://[^"]*{network}\.com[^"]*)"')
                network_links = pattern_network.findall(contenu_html)
                social_network_links[network] = set(network_links)

            contact_inf = ContactItem(
                    url= _url,
                    emails= list(_emails),
                    phones= list(telephones),
                    facebook= list(social_network_links["facebook"]),
                    twitter= list(social_network_links["twitter"]),
                    instagram= list(social_network_links["instagram"]),
                    linkedin= list(social_network_links["linkedin"]),
                    youtube= list(social_network_links["youtube"]),
                    tiktok= list(social_network_links["tiktok"])
                )

            result = {}
            result["status"] = "success"
            result["value"] = contact_inf
            return result

        except Exception as e:
            if contact_link_inf :
                error_message = f"Error :\n==>>>(in: extract_contact_information() at ContactProcessing)\n==>>>(current_url_contact = {_url})\n==>>>(select contact with = {contact_link_inf})\n==>>> {e.args[0]}"
            else:
                error_message = f"Error :\n==>>>(in: extract_contact_information() at ContactProcessing)\n==>>> {e.args[0]}"
                
            print(error_message)
            result = {}
            result["status"] = "error"
            result["value"] = error_message
            return error_message
                        
    def extract_contact_links(self, contenu_html):
        pattern_href = re.compile(r'<a .*href="([^"]*[cC][oO][nN][tT][aA][cC][tT][^"]*)"')
        liens_trouves_href = pattern_href.findall(contenu_html)

        if liens_trouves_href:
            return liens_trouves_href
        else:
            terms_related_to_contact = ['contact', 'contactez-nous','Contactez nous', 'nous contacter', 'الإتصال', 'Qui sommes-nous', 'اتصل بنا',
                "Contact",
                "Contactez-nous",
                "Contactez nous",
                "Nous contacter",
                "Nous joindre",
                "Pour nous contacter",
                "Coordonnées",
                "Informations de contact",
                "Service client",
                "Support",
                "Aide",
                "Formulaire de contact",
                "Adresse",
                "Page de contact",
                "Qui sommes-nous",
                "Qui sommes nous",
                "Rejoignez-nous",
                "Travaillez avec nous",
                "الإتصال",
                "تواصل معنا",
                "اتصل بنا",
                "تواصل",
                "تواصلوا معنا",
                "معلومات الاتصال",
                "خدمة العملاء",
                "الدعم",
                "المساعدة",
                "نموذج الاتصال",
                "العنوان",
                "صفحة الاتصال",
                "من نحن",
                "الأسئلة الشائعة",
                "نشرة إخبارية",
                "انضم إلينا",
                "اعمل معنا",
                "Contact",
                "Contact us",
                "Get in touch",
                "Reach out to us",
                "Contact information",
                "Customer service",
                "Support",
                "Help",
                "Contact form",
                "Address",
                "Contact page",
                "About",
                "About us",
                "Join us",
                "Work with us"
                ]
            
            pattern_text = re.compile(r'<a [^>]*href="([^"]*)"[^>]*>.*({}).*<\/a>'.format('|'.join(terms_related_to_contact)), re.IGNORECASE)
            liens_trouves_text = pattern_text.findall(contenu_html)
            return liens_trouves_text

In [4]:
class ContactItem:
    def __init__(self, url="", emails=None, phones=None, facebook=None, twitter=None, instagram=None, linkedin=None, youtube=None, tiktok=None, contact_sources=None):
        self.url = url
        self.emails = emails or []
        self.phones = phones or []
        self.facebook = facebook or []
        self.twitter = twitter or []
        self.instagram = instagram or []
        self.linkedin = linkedin or []
        self.youtube = youtube or []
        self.tiktok = tiktok or []
        self.contact_sources = contact_sources or []
        
    def __str__(self):
        return f"URL: {self.url}\n" \
               f"Emails: {', '.join(self.emails)}\n" \
               f"Phones: {', '.join(self.phones)}\n" \
               f"Facebook: {', '.join(self.facebook)}\n" \
               f"Twitter: {', '.join(self.twitter)}\n" \
               f"Instagram: {', '.join(self.instagram)}\n" \
               f"LinkedIn: {', '.join(self.linkedin)}\n" \
               f"YouTube: {', '.join(self.youtube)}\n" \
               f"Tiktok: {', '.join(self.tiktok)}\n" \
               f"Contact_sources: {', '.join(self.contact_sources)}"

    def concatenate_lists(self, other_contact):
        self.emails = list(set(self.emails + other_contact.emails))
        self.phones = list(set(self.phones + other_contact.phones))
        self.facebook = list(set(self.facebook + other_contact.facebook))
        self.twitter = list(set(self.twitter + other_contact.twitter))
        self.instagram = list(set(self.instagram + other_contact.instagram))
        self.linkedin = list(set(self.linkedin + other_contact.linkedin))
        self.youtube = list(set(self.youtube + other_contact.youtube))
        self.tiktok = list(set(self.tiktok + other_contact.tiktok))
        self.contact_sources = list(set(self.contact_sources + other_contact.contact_sources))

    def is_empty(self):    
        """
        This function checks if the object has any non-empty attributes.

        Args:
        obj: The object to be checked.

        Returns:
        True if all attributes are empty, False otherwise.
        """
        # Check each attribute if it's empty or not
        return ((self.emails is None or not self.emails) and
            (self.phones is None or not self.phones) and
            (self.facebook is None or not self.facebook) and
            (self.twitter is None or not self.twitter) and
            (self.instagram is None or not self.instagram) and
            (self.linkedin is None or not self.linkedin) and
            (self.youtube is None or not self.youtube) and
            (self.tiktok is None or not self.tiktok) and
            (self.contact_sources is None or not self.contact_sources))
    
    def add_contact_sources(self):
        if self.is_empty():
            self.contact_sources = [f"{self.url} ==> {not self.is_empty()}"]
        else:
            self.contact_sources = [f"{self.url} ==> {not self.is_empty()}"]

In [5]:
import threading
import csv

class ContactStorage:
    def __init__(self, file_path):
        self.file_path = f"{file_path}.csv"
        self.fieldnames = ['url', 'emails', 'telephones', 'facebook', 'twitter', 'instagram', 'linkedin', 'youtube', 'tiktok', 'contact_sources']
        self.file = open(self.file_path, 'a', newline='', encoding='utf-8')
        self.writer = csv.DictWriter(self.file, fieldnames=self.fieldnames)
        self.lock_file = threading.Lock()
        
        # Check if the file is empty, then write the header
        if self.file.tell() == 0:
            self.writer.writeheader()
            
        ################################################## for : Errors file ##############################
        self.file_errors = open(f"{file_path}_errors.csv", 'a', newline='', encoding='utf-8')
        self.writer_errors = csv.DictWriter(self.file_errors, fieldnames=["url","error_message"])
        self.lock_file_errors = threading.Lock()
        # Check if the file is empty, then write the header
        if self.file_errors.tell() == 0:
            self.writer_errors.writeheader()

    def insert_contact(self, contact_item):
        contact_data = {
            'url': contact_item.url,
            'emails': ', '.join(contact_item.emails),
            'telephones': ', '.join(contact_item.phones),
            'facebook': ', '.join(contact_item.facebook),
            'twitter': ', '.join(contact_item.twitter),
            'instagram': ', '.join(contact_item.instagram),
            'linkedin': ', '.join(contact_item.linkedin),
            'youtube': ', '.join(contact_item.youtube),
            'tiktok': ', '.join(contact_item.tiktok),
            'contact_sources': ', '.join(contact_item.contact_sources)
        }

        self.writer.writerow(contact_data)

    def close_file(self):
        self.file.close()
        
    ################################################## for : Errors file ##############################
    def write_error(self, url, error_message):
        error_content = {
            'url': url,
            'error_message': error_message
            }
        
        self.writer_errors.writerow(error_content)
        
    def close_file_errors(self):
        self.file_errors.close()

In [6]:
import threading
import pandas as pd
import csv
import time
import re
import os

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities

#
# import classes
#

class WorkerThread(threading.Thread):
    
    def __init__(self, thread_id, shared_list, contact_storage):
        super(WorkerThread, self).__init__()
        self.thread_id = thread_id
        self.shared_list = shared_list
        self.contact_storage = contact_storage
        
        # IP address and port and server of the Selenium hub and browser options
        self.HUB_HOST = "192.168.3.32"
        self.HUB_PORT = 4444
        self.server = f"http://{self.HUB_HOST}:{self.HUB_PORT}/wd/hub"
        self.options = webdriver.ChromeOptions()
        self.driver = None
        self.contactProcessing = ContactProcessing()
        
    def run(self):
        while True:
            try:
                # Access the shared list synchronously
                with self.shared_list.lock:
                    if not self.shared_list.data:
                        break  # The list is empty, the thread ends

                    url = self.shared_list.data.pop(0)

                    print(f"%"*50)
                    print(f"url_item = {url} |||| by_thread_id = {self.thread_id}")
                    print(f"%"*50)
                    
                self.driver = webdriver.Remote(command_executor=self.server, options=self.options)
                
                contact_item = self.scraping_contact(url)
                
                if contact_item :
                    # Store result in CSV file synchronously
                    with self.contact_storage.lock_file:
                        self.contact_storage.insert_contact(contact_item)

            except IndexError:
                pass  # La liste était probablement vide, pas besoin de gérer l'exception ici ???????????????????

            
    def scraping_contact(self, _url):
        try:
            self.driver.get(_url)
            WebDriverWait(self.driver, 10).until(
                EC.presence_of_element_located((By.TAG_NAME, "body"))
             )
            _url = self.driver.current_url
            contact_item = ContactItem(url=_url)
            
            contenu_html = self.driver.page_source
            
            contact_inf_result_0 = self.contactProcessing.extract_contact_information(contenu_html, _url)
            
            if contact_inf_result_0["status"] == "success":
                
                contact_inf_result_0["value"].add_contact_sources()
                contact_item.concatenate_lists(contact_inf_result_0["value"])

                contact_links = self.contactProcessing.extract_contact_links(contenu_html)

                i=0
                for contact_link in contact_links:
                    try:
                        if i != 0:
                            self.driver.get(_url)
                            WebDriverWait(self.driver, 10).until(
                                EC.presence_of_element_located((By.TAG_NAME, "body"))
                             )

                        self.go_to_contact_page(contact_link, contact_links)
                        WebDriverWait(self.driver, 10).until(
                                        EC.presence_of_element_located((By.TAG_NAME, "body"))
                                     )
                        
                        contenu_html = self.driver.page_source
                        current_url_contact = self.driver.current_url

                        contact_inf_result_1 = self.contactProcessing.extract_contact_information(contenu_html, current_url_contact, contact_link)
                        
                        if contact_inf_result_1["status"] == "success" :
                            
                            contact_inf_result_1["value"].add_contact_sources()
                            contact_item.concatenate_lists(contact_inf_result_1["value"])
                        else:
                            with self.contact_storage.lock_file_errors:
                                self.contact_storage.write_error(_url, contact_inf_result_1["value"])

                    except Exception as e:
                        error_message = f"==>>> Error : (in: for contact_link in contact_links:)\n(current_url_contact = {current_url_contact})\n(select contact with = {contact_link}) =>>> {e.args[0]}"
                        print(error_message)
                        
                        with self.contact_storage.lock_file_errors:
                            self.contact_storage.write_error(_url, error_message)
                        
                    finally:
                        i=i+1

                print(contact_item)
                print(f"_______________By : {self.thread_id}___________________________________________________________________________\n\n")

                return contact_item
            
            else:
                with self.contact_storage.lock_file_errors:
                    self.contact_storage.write_error(_url, contact_inf_result_0["value"]) # ?????????????????????????????????????????????????????????
                
        except Exception as e:
            error_message = f"Error :\n==>>>(in: scraping_contact() at WorkerThread)\n==>>> {e.args[0]}"
            print(error_message)
            
            with self.contact_storage.lock_file_errors:
                self.contact_storage.write_error(_url, error_message)
            
        finally:
            self.driver.quit()
            
    def go_to_contact_page(self, contact_link, contact_links):
        link_elements_href = self.driver.find_elements(By.CSS_SELECTOR, f'[href="{contact_link}"]')
        if link_elements_href and len(link_elements_href) > 0:
            link_element_href = link_elements_href[0]
            self.driver.execute_script("arguments[0].click();", link_element_href)
            mode = "href_1"
        else:
            link_elements_href = self.driver.find_elements(By.CSS_SELECTOR, f'[href="{contact_link[0]}"]')
            if link_elements_href and len(link_elements_href) > 0:
                link_element_href = link_elements_href[0]
                self.driver.execute_script("arguments[0].click();", link_element_href)
                mode = "href_2"
            else:
                link_elements_text = self.driver.find_elements(By.XPATH, f'//a[contains(text(), "{contact_link[1]}")]')
                if link_elements_text and len(link_elements_text) > 0:
                    link_element_text = link_elements_text[0]
                    self.driver.execute_script("arguments[0].click();", link_element_text)
                    mode = "text"
                else:
                    print(f"@@@"*50)
                    print(f"contact_links = {contact_links}")
                    print(f":( :( :( ==> link : {contact_link[1]}")
                    print(f"@@@"*50)
                    
        print(f"$$$"*50)
        print(f"mode = {mode} || contact_link = {contact_link}")
        print(f"$$$"*50)
        time.sleep(1)

In [10]:
################################################ main ###########################################################
#################################################################################################################
import threading
import pandas as pd
import csv
import time
import re
import os

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities

os.makedirs("results", exist_ok=True)

# Créer une liste partagée, une file de résultats et un verrou CSV
shared_list = SharedListURLs()
shared_list.load_data()

# Créer et démarrer les threads
num_threads = 5

start = 0
end = min(20,shared_list.nb_URLs)
step = 20

for i in range(start, end, step):
    
    shared_list.select_data(i,i+step)
    
    contact_storage = ContactStorage('results/contacts')
    
    threads = []
    for j in range(num_threads):
        thread = WorkerThread(j+1, shared_list, contact_storage)
        thread.start()
        threads.append(thread)

    # Attendre que tous les threads se terminent
    for thread in threads:
        thread.join()

    # Fermer les fichiers
    contact_storage.close_file()
    contact_storage.close_file_errors()
    
    print(f"%"*100)
    print("saved URLs : ",i+step)
    print(f"%"*100)
    
    time.sleep(5)

%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
url_item = http://hespress.com/ |||| by_thread_id = 1
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
url_item = https://www.hibapress.com |||| by_thread_id = 2
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
url_item = https://www.goud.ma |||| by_thread_id = 3
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
url_item = http://lemonde.fr/ |||| by_thread_id = 4
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
url_item = https://www.lematin.ma |||| by_thread_id = 5
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
URL: https://hespress.com/
Emails: 
Phones: 
Facebook: 
Twitter: 
Instagram: 
LinkedIn: 
YouTube: 
Tiktok: 
Contact_sources: https://hespress.com/ ==> False
_______________By : 1______________________________________

In [7]:

# Créer une liste partagée, une file de résultats et un verrou CSV
shared_list = SharedListURLs()
shared_list.load_data()

# Créer et démarrer les threads
num_threads = 4

start = 0
end = min(20,shared_list.nb_URLs)
step = 5

In [8]:
shared_list.nb_URLs

2645

In [ ]:
ana : 1000-1800
morad : 1800-2645

In [2]:
for x in range(5):
    try:
        if x == 2:
            raise ValueError("x doit être positif")
        print(f"x = {x}")
    except ValueError as e:
        print(f"Erreur : {e}")

x = 0
x = 1
Erreur : x doit être positif
x = 3
x = 4
